Cell 1: Setup the environment


Load environment variables and suppress warnings to ensure the environment is properly set up for the LangChain chains. Additionally, the code handles model deprecation by selecting the correct LLM model based on the current date.



In [4]:
# Import necessary modules
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables from a .env file
_ = load_dotenv(find_dotenv())  # Load .env file

# Suppress warnings related to deprecated functions or modules
import warnings
warnings.filterwarnings('ignore')

# Handle model deprecation based on current date
import datetime

# Get the current date
current_date = datetime.datetime.now().date()

# Define a target date after which a new model should be used
target_date = datetime.date(2024, 6, 12)

# Conditionally set the LLM model based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"


Cell 2: Load and preview data


Load a CSV file (Data.csv) using Pandas and preview the first few rows. This data is used later for sequential chains involving review translation.



In [ ]:
# Install pandas if not already installed
#!pip install pandas

# Import pandas
import pandas as pd

# Load the dataset
df = pd.read_csv('Data.csv')

# Display the first few rows of the dataset
df.head()


Cell 3: LLMChain example


Demonstrate the use of LLMChain in LangChain by generating a product name suggestion for a given input using a language model.

In [ ]:
# Import necessary classes for LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

# Initialize the LLM with specified model and temperature
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# Define a prompt template that asks for a company name based on a product
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

# Create the LLMChain with the prompt and the LLM
chain = LLMChain(llm=llm, prompt=prompt)

# Define the product
product = "Queen Size Sheet Set"

# Run the chain with the product input
chain.run(product)


Cell 4: SimpleSequentialChain example

Use SimpleSequentialChain to create a chain of two LLMs where the output of the first is passed as input to the second.



In [ ]:
# Import SimpleSequentialChain class
from langchain.chains import SimpleSequentialChain

# Initialize the LLM with specified model and temperature
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# Define the first prompt template (company name suggestion)
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

# Chain 1: Suggest company name
chain_one = LLMChain(llm=llm, prompt=first_prompt)

# Define the second prompt template (company description)
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company: {company_name}"
)

# Chain 2: Provide a description for the suggested company name
chain_two = LLMChain(llm=llm, prompt=second_prompt)

# Combine the two chains into a SimpleSequentialChain
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two], verbose=True)

# Run the combined chain with the product input
overall_simple_chain.run(product)


Cell 5: SequentialChain example


Create a SequentialChain that chains multiple LLMs together, processing inputs and outputs across multiple steps.



In [ ]:
# Import SequentialChain class
from langchain.chains import SequentialChain

# Initialize the LLM
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# First chain: Translate review to English
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to English:\n\n{Review}"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")

# Second chain: Summarize the English review
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:\n\n{English_Review}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

# Third chain: Detect the language of the review
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

# Fourth chain: Write a follow-up response in the detected language
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow-up response to the following summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="followup_message")

# Combine all chains into a SequentialChain
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True
)

# Get a review from the dataset and run the chain
review = df.Review[5]
overall_chain(review)


Cell 6: Router Chain example

Use a RouterChain to dynamically choose between different LLM chains based on the input type (e.g., physics, math, history, or computer science questions).



In [ ]:
# Define prompt templates for different subject areas
physics_template = """You are a very smart physics professor..."""
math_template = """You are a very good mathematician..."""
history_template = """You are a very good historian..."""
computerscience_template = """You are a successful computer scientist..."""

# Define a list of prompt information
prompt_infos = [
    {"name": "physics", "description": "Good for answering questions about physics", "prompt_template": physics_template},
    {"name": "math", "description": "Good for answering math questions", "prompt_template": math_template},
    {"name": "History", "description": "Good for answering history questions", "prompt_template": history_template},
    {"name": "computer science", "description": "Good for answering computer science questions", "prompt_template": computerscience_template}
]

# Import necessary classes for the router chain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

# Initialize the LLM
llm = ChatOpenAI(temperature=0, model=llm_model)

# Create destination chains for each prompt
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

# Define the default prompt for unhandled cases
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

# Define the routing logic template
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input..."""
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations="\n".join([f"{p['name']}: {p['description']}" for p in prompt_infos]))
router_prompt = PromptTemplate(template=router_template, input_variables=["input"], output_parser=RouterOutputParser())

# Initialize the router chain
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

# Create the final multi-prompt chain
chain = MultiPromptChain(router_chain=router_chain, destination_chains=destination_chains, default_chain=default_chain, verbose=True)

# Test the router chain with different inputs
chain.run("What is black body radiation?")
chain.run("What is 2 + 2?")
chain.run("Why does every cell in our body contain DNA?")
